# Resume Experiment Analysis

How much harder is it to get a job in the United States if you are Black than if you are White? Or, expressed differently, what is the *effect* of race on the difficulty of getting a job in the US?

In this exercise, we will be analyzing data from a real world experiment designed to help answer this question. Namely, we will be analyzing data from a randomized experiment in which 4,870 ficticious resumes were sent out to employers in response to job adverts in Boston and Chicago in 2001. The resumes differ in various attributes including the names of the applicants, and different resumes were randomly allocated to job openings. 

The "experiment" part of the experiment is that resumes were randomly assigned Black- or White-sounding names, and then watched to see whether employers called the "applicants" with Black-sounding names at the same rate as the applicants with the White-sounding names.

(Which names constituted "Black-sounding names" and "White-sounding names" was determined by analyzing names on Massachusetts birth certificates to determine which names were most associated with Black and White children, and then surveys were used to validate that the names were perceived as being associated with individuals of one racial category or the other. Also, please note I subscribe to the logic of [Kwame Anthony Appiah](https://www.theatlantic.com/ideas/archive/2020/06/time-to-capitalize-blackand-white/613159/) and chose to capitalize both the B in Black and the W in White). 

You can get access to original article [here](https://www.aeaweb.org/articles?id=10.1257/0002828042002561). 

**Note to Duke students:** if you are on the Duke campus network, you'll be able to access almost any academic journal articles directly; if you are off campus and want access, you can just go to the [Duke Library](https://library.duke.edu/) website and search for the article title. Once you find it, you'll be asked to log in, after which you'll have full access to the article. You will also find this pattern holds true at nearly any major University in the US.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/ids720_specific/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_resume_experiment.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex2_pvalue_computerskills",
    "ex2_pvalue_female",
    "ex2_pvalue_yearsexp",
    "ex3_pvalue_education",
    "ex4_validity",
    "ex5_pvalue",
    "ex5_white_advantage_percent",
    "ex5_white_advantage_percentage_points",
    "ex6_black_pvalue",
    "ex8_black_college",
    "ex8_black_nocollege",
    "ex8_college_heterogeneity",
    "ex9_gender_and_discrimination",
    "ex10_experiment_v_us",
}
```


### Submission Limits

Please remember that you are **only allowed THREE submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Checking for Balance

The first step in analyzing any experiment is to check whether you have *balance* across your treatment arms—that is to say, do the people who were randomly assigned to the treatment group look like the people who were randomly assigned to the control group. Or in this case, do the resumes that ended up with Black-sounding names look like the resumes with White-sounding names. 

Checking for balance is critical for two reasons. First, it's always possible that random assignment will create profoundly different groups—the *Large of Large Numbers* is only a "law" in the limit. So we want to make sure we have reasonably similar groups from the outset. And second, it's also always possible that the randomization wasn't actually implemented correctly—you would be amazed at the number of ways that "random assignment" can go wrong! So if you ever do find you're getting unbalanced data, you should worry not only about whether the groups have baseline differences, but also whether the "random assignment" was actually random!

### Exercise 1

Download the data set from this experiment (`resume_experiment.dta`) from [github](https://github.com/nickeubank/MIDS_Data/tree/master/resume_experiment). To aid the autograder, please load the data directly from a URL.


In [1]:
import pandas as pd
import numpy as np

db_resume = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/"
    "raw/refs/heads/master/resume_experiment/"
    "resume_experiment.dta"
)
print(db_resume.columns)
db_resume.sample(2)

Index(['education', 'ofjobs', 'yearsexp', 'computerskills', 'call', 'female',
       'black'],
      dtype='object')


,education,ofjobs,yearsexp,computerskills,call,female,black
2931,4,4,11,1,0.0,1.0,1.0
1652,3,3,8,1,0.0,1.0,0.0



### Exercise 2

- `black` is the treatment variable in the data set (whether the resume has a "Black-sounding" name).
- `call` is the dependent variable of interest (did the employer call the fictitious applicant for an interview)

In addition, the data include a number of variables to describe the other features in each fictitious resume, including applicants education level (`education`), years of experience (`yearsexp`), gender (`female`), computer skills (`computerskills`), and number of previous jobs (`ofjobs`). Each resume has a random selection of these attributes, so on average the Black-named fictitious applicant resumes have the same qualifications as the White-named applicant resumes. 

Check for balance in terms of the average values of applicant gender (`female`), computer skills (`computerskills`), and years of experience (`yearsexp`) across the two arms of the experiment (i.e. by `black`). Calculate both the differences in means across treatment arms *and* test for statistical significance of these differences. Does gender, computer skills, and yearsexp look balanced across race groups in terms of both statistical significance and magnitude of difference?

Store the p-values associated with your t-test of these variables in `ex2_pvalue_female`, `ex2_pvalue_computerskills`, and `ex2_pvalue_yearsexp`. **Round your values to 2 decimal places.**


In [ ]:
from scipy import stats

# mean values for female by race group
mean_female_black = db_resume[db_resume["black"] == 1]["female"].mean()
mean_female_white = db_resume[db_resume["black"] == 0]["female"].mean()
diff_female = mean_female_white - mean_female_black
print(
    f"The differences in means of applicant gender across the two arms of the experiment is {diff_female}."
)

# mean values for computerskills by race group
mean_computerskills_black = db_resume[db_resume["black"] == 1]["computerskills"].mean()
mean_computerskills_white = db_resume[db_resume["black"] == 0]["computerskills"].mean()
diff_computerskills = mean_computerskills_white - mean_computerskills_white
print(
    f"The differences in means of applicant computer skills across the two arms of the experiment is {diff_computerskills}."
)

# mean values for yearsexp by race group
mean_yearsexp_black = db_resume[db_resume["black"] == 1]["yearsexp"].mean()
mean_yearsexp_white = db_resume[db_resume["black"] == 0]["yearsexp"].mean()
diff_yearsexp = mean_yearsexp_white - mean_yearsexp_black
print(
    f"The differences in means of applicant years of experience across the two arms of the experiment is {diff_yearsexp} years."
)

# t-tests for female
t_female, p_female = stats.ttest_ind(
    db_resume[db_resume["black"] == 1]["female"],
    db_resume[db_resume["black"] == 0]["female"],
)
significance_female = "significant" if p_female <= 0.05 else "insignificant"
print(
    f"The p-value of the differences in means of applicant gender across the two arms of the experiment is {p_female}."
)
print(
    f"This indicates the differences in means of applicant gender across the two arms of the experiment is {significance_female}."
)
# t-tests for computerskills
t_computerskills, p_computerskills = stats.ttest_ind(
    db_resume[db_resume["black"] == 1]["computerskills"],
    db_resume[db_resume["black"] == 0]["computerskills"],
)
significance_computerskills = (
    "significant" if p_computerskills <= 0.05 else "insignificant"
)
print(
    f"The p-value of the differences in means of applicant computer skills across the two arms of the experiment is {p_computerskills}."
)
print(
    f"This indicates the differences in means of applicant computer skills across the two arms of the experiment is {significance_computerskills}."
)

# t-tests for yearsexp
t_yearsexp, p_yearsexp = stats.ttest_ind(
    db_resume[db_resume["black"] == 1]["yearsexp"],
    db_resume[db_resume["black"] == 0]["yearsexp"],
)
significance_yearsexp = "significant" if p_yearsexp <= 0.05 else "insignificant"
print(
    f"The p-value of the differences in means of applicant years of experience across the two arms of the experiment is {p_yearsexp}."
)
print(
    f"This indicates the differences in means of applicant years of experience across the two arms of the experiment is {significance_yearsexp}."
)

# results
results = {
    "ex2_pvalue_female": round(p_female, 2),
    "ex2_pvalue_computerskills": round(p_computerskills, 2),
    "ex2_pvalue_yearsexp": round(p_yearsexp, 2),
}

print(f'ex2_pvalue_female: {results["ex2_pvalue_female"]}')
print(f'ex2_pvalue_computerskills: {results["ex2_pvalue_computerskills"]}')
print(f'ex2_pvalue_yearsexp: {results["ex2_pvalue_yearsexp"]}')

The differences in means of applicant gender across the two arms of the experiment is -0.010677635669708252.
The differences in means of applicant computer skills across the two arms of the experiment is 0.0.
The differences in means of applicant years of experience across the two arms of the experiment is 0.026694045174538772 years.
The p-value of the differences in means of applicant gender across the two arms of the experiment is 0.37666855949097355.
This indicates the differences in means of applicant gender across the two arms of the experiment is insignificant.
The p-value of the differences in means of applicant computer skills across the two arms of the experiment is 0.03032693395539194.
This indicates the differences in means of applicant computer skills across the two arms of the experiment is significant.
The p-value of the differences in means of applicant years of experience across the two arms of the experiment is 0.8535350182481283.
This indicates the differences in mean

>- For gender,
>   - the differences in means is statistical insignificance. 
>   - the magnitude of difference is 0.011.
>- For computer skills,
>   - the differences in means is statistical significance. 
>   - the magnitude of difference is 0.0.
>- For years of experience,
>   - the differences in means is statistical insignificance. 
>   - the magnitude of difference is 0.0267.

### Exercise 3

Do a similar tabulation for education (`education`). Education is a categorical variable coded as follows:

- 0: Education not reported
- 1: High school dropout
- 2: High school graduate
- 3: Some college
- 4: College graduate or higher

Because these are categorical, you shouldn't just calculate and compare means—you should compare share or count of observations with each value (e.g., a chi-squared contingency table). You may also find the `pd.crosstab` function useful.

Does education look balanced across racial groups?

Store the p-value from your chi squared test in results under the key `ex3_pvalue_education`. **Please round to 2 decimal places.**

In [ ]:
# Cross-tabulate education by race group
education_crosstab = pd.crosstab(db_resume["education"], db_resume["black"])
print(education_crosstab)

# Perform Chi-Square test
chi2, p_education, _, _ = stats.chi2_contingency(education_crosstab)
significance_education = "significant" if p_education <= 0.05 else "insignificant"
print(f"chi^2 is {chi2}")
print(f"The p-value of the chi squared test is {p_education}.")
print(
    f"This indicates the differences of applicant education across the two arms of the experiment is {significance_education}."
)

# results
results["ex3_pvalue_education"] = round(p_education, 2)
print(f'ex3_pvalue_education: {results["ex3_pvalue_education"]}')

black       0.0   1.0
education            
0            18    28
1            18    22
2           142   132
3           513   493
4          1744  1760
chi^2 is 3.4095502219737974
The p-value of the chi squared test is 0.4917640058792273.
This indicates the differences of applicant education across the two arms of the experiment is insignificant.
ex3_pvalue_education: 0.49


>Education looks balanced across racial groups as the difference is statistically insignificant.

### Exercise 4

What do you make of the overall results on resume characteristics? Why do we care about whether these variables look similar across the race groups? And if they didn't look similar, would that be a threat to internal or external validity? 

Answer in markdown, then also store your answer to the question of whether imbalances are a threat to internal or external validity in `"ex4_validity"` as the string `"internal"` or `"external"`.


>Data balance between the two arms of the experiment is fundamental to data analysis. If the data is imbalanced, even if the selection is truly random, the result might turn out to be different, which indicates that the assumption of "No Baseline Differences" between the treatment and control groups is not met. This concern refers to internal validity, as we are interested in knowing how well our predictions fit the data we already have.

In [4]:
results["ex4_validity"] = "internal"
print(f'ex3_pvalue_education: {results["ex4_validity"]}')

ex3_pvalue_education: internal


## Estimating Effect of Race

### Exercise 5

The variable of interest in the data set is the variable `call`, which indicates a call back for an interview. Perform a two-sample t-test comparing applicants with black sounding names and white sounding names.

Interpret your results—in both percentage *and* in percentage points, what is the effect of having a Black-sounding name (as opposed to a White-sounding name) on your resume?

Store how much more likely a White applicant is to receive a call back than a Black respondent in percentage and percentage points in `"ex5_white_advantage_percent"`and `"ex5_white_advantage_percentage_points"`. Please scale percentages so 1 is 1% and percentage points so a value of `1` corresponds to 1 percentage point. **Please round these answers to 2 decimal places.**

Store the p-value of the difference in `"ex5_pvalue"` **Please round your p-value to 5 decimal places.**

In [ ]:
# t-test for the call variable
t_call, p_call = stats.ttest_ind(
    db_resume[db_resume["black"] == 1]["call"],
    db_resume[db_resume["black"] == 0]["call"],
)
significance_call = "significant" if p_call <= 0.05 else "insignificant"

# white advantage - call percentage
call_rate_black = db_resume[db_resume["black"] == 1]["call"].mean()
call_rate_white = db_resume[db_resume["black"] == 0]["call"].mean()

white_advantage_percent = round(
    (call_rate_white - call_rate_black) / call_rate_black * 100, 2
)
white_advantage_percentage_points = round((call_rate_white - call_rate_black) * 100, 2)

print(
    f"The advantage of having a White-sounding name over a Black-sounding name in terms of callback percentage is: {white_advantage_percent:.2f}%"
)
print(
    f"The advantage of having a White-sounding name over a Black-sounding name in terms of percentage points is: {white_advantage_percentage_points:.2f} percentage points"
)
print(
    f"The p-value of the difference in callback rates between applicants with Black-sounding names and White-sounding names is: {p_call:.5f}"
)
print(
    f"This indicates that the difference in callback rates between applicants with Black-sounding names and those with White-sounding names is {significance_call}."
)

# Results
results["ex5_white_advantage_percent"] = white_advantage_percent
results["ex5_white_advantage_percentage_points"] = white_advantage_percentage_points
results["ex5_pvalue"] = round(p_call, 5)

print(f'ex5_white_advantage_percent: {results["ex5_white_advantage_percent"]}')
print(
    f'ex5_white_advantage_percentage_points: {results["ex5_white_advantage_percentage_points"]}'
)
print(f'ex5_pvalue: {results["ex5_pvalue"]}')

The advantage of having a White-sounding name over a Black-sounding name in terms of callback percentage is: 49.68%
The advantage of having a White-sounding name over a Black-sounding name in terms of percentage points is: 3.20 percentage points
The p-value of the difference in callback rates between applicants with Black-sounding names and White-sounding names is: 0.00004
This indicates that the difference in callback rates between applicants with Black-sounding names and those with White-sounding names is significant.
ex5_white_advantage_percent: 49.68
ex5_white_advantage_percentage_points: 3.2
ex5_pvalue: 4e-05


>The result indicates that the statistical significance in difference in callback rates between applicants with Black-sounding names and those with White-sounding names. Specifically, applicants with White-sounding names have a higher callback rate than applicants with Black-sounding names.

### Exercise 6

Now, use a linear probability model (a linear regression with a 0/1 dependent variable!) to estimate the differential likelihood of being called back by applicant race (i.e. the racial discrimination by employers). Please use [statsmodels](https://www.statsmodels.org/stable/index.html).

Since we have a limited dependent variable, be sure to use [heteroskedastic robust standard errors.](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLSResults.get_robustcov_results.html) Personally, I prefer the `HC3` implementation, as it tends to do better with smaller samples than other implementations.

Interpret these results—what is the *effect* of having a Black-sounding name (as opposed to a White-sounding name) on your resume in terms of the likelihood you'll be called back? 

How does this compare to the estimate you got above in exercise 5?

Store the p-value associated with `black` in `"ex6_black_pvalue"`. **Please round your pvalue to 5 decimal places.**

In [ ]:
import statsmodels.api as sm

# model
X = sm.add_constant(db_resume[["black"]])
y = db_resume["call"]

# Fit the model using robust standard errors
model = sm.OLS(y, X).fit(cov_type="HC3")

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     16.92
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           3.96e-05
Time:                        23:08:22   Log-Likelihood:                -562.24
No. Observations:                4870   AIC:                             1128.
Df Residuals:                    4868   BIC:                             1141.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0965      0.006     16.121      0.0

In [ ]:
# Result
results["ex6_black_pvalue"] = round(model.pvalues["black"], 5)
print(f"ex6_black_pvalue: {results["ex6_black_pvalue"]}")
print(
    f"In the linear probability model, the p-value associated with 'black' is {results["ex6_black_pvalue"]}."
)

ex6_black_pvalue: 4e-05
In the linear probability model, the p-value associated with 'black' is 4e-05.


>- The coefficient of variable 'black' is -0.0320 indicating that, on average, applicants with Black-sounding names have a lower probability of being called back by approximately 3.20 percentage points compared to those with white-sounding names.
>- The p-value associated with 'black' is 0.00004 indicating that the difference of probability of being called back between applicants with Black-sounding names and applicants with white-sounding names is statistically significant.
>- This p-value is the same as the one calculated in exercise 5.
>- All the numbers aligns to the conclusion that it it harder to be called back for applicants with black-sounding names than applicants with white-sounding names.

### Exercise 7

Even when doing a randomized experiment, adding control variables to your regression *can* improve the statistical efficiency of your estimates of the treatment effect (the upside is the potential to explain residual variation; the downside is more parameters to be estimated). Adding controls can be particularly useful when randomization left some imbalances in covariates (which you may have seen above). 

Now let's see if we can improve our estimates by adding in other variables as controls. Add in `education`, `yearsexp`, `female`, and `computerskills`—be sure to treat education as a categorical variable!

In [ ]:
db_resume["education"] = db_resume["education"].astype("category")
X_control = sm.add_constant(
    db_resume[["black", "education", "yearsexp", "female", "computerskills"]]
)
y = db_resume["call"]

import statsmodels.formula.api as smf

model_control = smf.ols(
    "call ~ black + C(education) + yearsexp + female + computerskills", data=db_resume
).fit(cov_type="HC3")

print(model_control.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     4.350
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           3.04e-05
Time:                        23:08:22   Log-Likelihood:                -551.02
No. Observations:                4870   AIC:                             1120.
Df Residuals:                    4861   BIC:                             1178.
Df Model:                           8                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0821      0.04

In [ ]:
print(
    f"In the revised model with more variables added, the coefficient associated with 'black' is {model_control.params["black"]}."
)
print(
    f"In the revised model with more variables added, the p-value associated with 'black' is {round(model_control.pvalues["black"], 5)}."
)

In the revised model with more variables added, the coefficient associated with 'black' is -0.031613719588877896.
In the revised model with more variables added, the p-value associated with 'black' is 5e-05.


>In the revised model,
>- the coefficient changed from -0.032 to -0.0316
>- the p-value that changed from 4e-05 to 5e-05
>
>This indicates that the result remains the same that it is harder to be called back for applicants with Black-sounding names than those with White-sounding names

## Estimating Heterogeneous Effects

### Exercise 8

As you may recall from some past readings (such as this one on the [migraine medication Aimovig](https://ds4humans.com/30_questions/15_answering_exploratory_questions.html#faithful-representations)), our focus on estimating *Average Treatment Effects* runs the risk of papering over variation in how individuals respond. In the case of Aimovig, for example, nearly no patients actually experienced the Average Treatment Effect of the medication; around half of patients experienced no benefit, while the other half experienced a benefit of about twice the average treatment effect.

So far in this analysis we've been focusing on the *average* effect of having a Black-sounding name (as compared to a White-sounding name). But we can actually use our regression framework to look for evidence of *heterogeneous treatment effects*—effects that are different for different types of people in our data. We accomplish this by *interacting* a variable we think may be related to experiencing a differential treatment effect with our treatment variable. For example, if we think that applicants with Black-sounding names who have a college degree are likely to experience less discrimination, we can interact `black` with an indicator for having a college degree. If having a college degree reduces discrimination, we could expect the interaction term to be positive. 

Is there more or less racial discrimination (the absolute magnitude difference in call back rates between Black and White applicants) among applicants who have a college degree? Store your answer as the string `"more discrimination"` or `"less discrimination"` under the key `"ex8_college_heterogeneity"`.

Please still include `education`, `yearsexp`, `female`, and `computerskills` as controls.

**Note:** it's relatively safe to assume that someone hiring employees who sees a resume that does *not* report education levels will assume the applicant does not have a college degree. So treat "No education reported" as "not having a college degree."

In percentage points, what is the difference in call back rates:

- between White applicants without a college degree and Black applicants without a college degree (`ex8_black_nocollege`).
- between White applicants with a college degree and Black applicants with a college degree (`ex8_black_college`).

Use negative values to denote a lower probability for Black applicants to get a call back. **Scale so a value of `1` is a one percentage point difference. Please round your answer to 2 percentage points.**

Focus on the coefficient values, even if the significance is low.

In [ ]:
# create a new column for college degree
db_resume["college_degree"] = (db_resume["education"] == 4).astype(int)

# Fit the model with interaction
model_interaction = smf.ols(
    "call ~ black*college_degree+ + black + college_degree + yearsexp + computerskills + female ",
    db_resume,
).fit(cov_type="HC3")
print(model_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     5.874
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           4.06e-06
Time:                        23:08:23   Log-Likelihood:                -550.77
No. Observations:                4870   AIC:                             1116.
Df Residuals:                    4863   BIC:                             1161.
Df Model:                           6                                         
Covariance Type:                  HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.0849 

In [ ]:
ex8_black_nocollege = model_interaction.params["black"] * 100
results["ex8_black_nocollege"] = round(ex8_black_nocollege, 2)
print(f"ex8_black_nocollege: {results["ex8_black_nocollege"]}")
print(
    f"The difference in call back rates between Black applicants without a college degree and White applicants without a college degree is {ex8_black_nocollege:.2f} percentage points.\n"
)

ex8_black_college = round(
    (
        model_interaction.params["black:college_degree"]
        + model_interaction.params["black"]
    )
    * 100,
    2,
)
results["ex8_black_college"] = round(ex8_black_college, 2)
print(f"ex8_black_college: {results["ex8_black_college"]}")

print(
    f"The difference in call back rates between Black applicants with a college degree and White applicants with a college degree is {ex8_black_college:.2f} percentage points."
)

ex8_black_nocollege: -4.05
The difference in call back rates between Black applicants without a college degree and White applicants without a college degree is -4.05 percentage points.

ex8_black_college: -2.81
The difference in call back rates between Black applicants with a college degree and White applicants with a college degree is -2.81 percentage points.


In [ ]:
ex8_college_heterogeneity = (
    "less discrimination"
    if ex8_black_nocollege < ex8_black_college
    else "more discrimination"
)
results["ex8_college_heterogeneity"] = ex8_college_heterogeneity
print(f"ex8_college_heterogeneity: {ex8_college_heterogeneity}")
print(
    f"Therefore, among applicants with a college degree, there is {ex8_college_heterogeneity} in call back rates."
)

ex8_college_heterogeneity: less discrimination
Therefore, among applicants with a college degree, there is less discrimination in call back rates.


### Exercise 9

Now let's compare men and women—is the penalty for having a Black-sounding name greater for Black men or Black women? Store your answer as `"greater discrimination for men"` or `"greater discrimination for women"` in `"ex9_gender_and_discrimination"`.

Focus on the coefficient values, even if the significance is low.

Again, please still include `education`, `yearsexp`, `female`, and `computerskills` as controls.

In [ ]:
# Fit the model with female interaction
model_gender_interaction = smf.ols(
    "call ~ black*female + black + C(education) + yearsexp + computerskills + female",
    db_resume,
).fit(cov_type="HC3")
print(model_gender_interaction.summary())

                            OLS Regression Results                            
Dep. Variable:                   call   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     3.866
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           6.76e-05
Time:                        23:36:30   Log-Likelihood:                -551.00
No. Observations:                4870   AIC:                             1122.
Df Residuals:                    4860   BIC:                             1187.
Df Model:                           9                                         
Covariance Type:                  HC3                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0807      0.04

In [ ]:
black_male = model_gender_interaction.params["black"] * 100
black_female = (
    model_gender_interaction.params["black"]
    + model_gender_interaction.params["black:female"]
) * 100

print(
    f"The difference in call back rates between Black male applicants and White male applicants is {black_male:.2f} percentage points.\n"
    f"The difference in call back rates between Black female applicants and White female applicants is {black_female:.2f} percentage points."
)

The difference in call back rates between Black male applicants and White male applicants is -2.87 percentage points.
The difference in call back rates between Black female applicants and White female applicants is -3.25 percentage points.


In [ ]:
ex9_gender_and_discrimination = (
    "greater discrimination for women"
    if black_male > black_female
    else "greater discrimination for men"
)
results["ex9_gender_and_discrimination"] = ex9_gender_and_discrimination
print(f"Therefore, there is {ex9_gender_and_discrimination} in call back rates.")

Therefore, there is greater discrimination for women in call back rates.


### Exercise 10

Calculate and/or lookup the following online:

- What is the share of applicants in our dataset with college degrees?
- What share of Black adult Americans have college degrees (i.e. have completed a bachelors degree)?

Is the share of Black applicants with college degrees in this data `"greater"`, or `"less"` than in the US? Store your answer as one of those strings in `"ex10_experiment_v_us"`

In [ ]:
# Share of applicants with a college degree
college_degree_share = db_resume["college_degree"].mean()
black_college_degree_share = db_resume[db_resume["black"] == 1]["college_degree"].mean()

print(
    f"The share of applicants in our dataset with college degrees is: {college_degree_share:.1%}"
)
print(
    f"The share of Black applicants in our dataset with college degrees is: {black_college_degree_share:.1%}"
)

results["ex10_experiment_v_us"] = (
    "greater" if college_degree_share < black_college_degree_share else "less"
)
print(f"ex10_experiment_v_us: {results["ex10_experiment_v_us"]}")
print(
    f"After researching online, we can conclude that the share of Black applicants with college degrees in this data is {results['ex10_experiment_v_us']} than in the US."
)

The share of applicants in our dataset with college degrees is: 72.0%
The share of Black applicants in our dataset with college degrees is: 72.3%
ex10_experiment_v_us: greater
After researching online, we can conclude tha the share of Black applicants with college degrees in this data is greater than in the US.


### Exercise 11

Bearing in mind your answers to Exercise 8 and to Exercise 10, how do you think the Average Treatment Effect you estimated in Exercises 5 and 6 might generalize to the experience of the average Black American (i.e., how do you think the ATE for the average Black American would compare to the ATE estimated from this experiment)?


>According to the results I got in Exercises 8 and 10, I think that the Average Treatment Effect estimated in Exercises 5 and 6 does not generalize the experience of the average Black American for the following reasons:
>- The dataset shows an artificially high proportion of Black individuals with college degrees compared to the actual share of Black adult Americans with such qualifications.
>- Observation shows that the discrimination effect against Black individuals in callback rates is lower among those with college degrees.
>
>Therefore, we could conclude that the ATE for the average Black American should be higher compared to the ATE estimated in this experiment, which is artificially reduced due to the disproportionately high proportion of individuals with college degrees in our sample.

### Exercise 12

What does your answer to Exercise 10 imply about the study's *internal* validity?

>I observed that the distribution of data in the experiment samples is balanced. The proportion of variables such as gender, educational level, and computer skills is distributed similarly for both groups, so we can consider that the study has good internal validity. However, it is important to consider the limitations of the study, as it only measures the effect of having a Black-sounding name.

### Exercise 13

What does your answer to Exercise 10 imply about the study's *external* validity?

>The study has some limitations in terms of external validity since it cannot be extended to other populations. That said, it cannot fully represent the reality of the experience of a Black person in the United States. It is important to consider that external validity depends on the population we wish to compare it with. However, if the goal is to generalize to the Black population of the United States, this experiment has limited external validity, since the sample we are using differs significantly from the actual population. Furthermore, it does not take into account all the differences that may influence a person's experience in the labor market, beyond the impact of the name, whether it sounds like a Black name or not.

## What Did We Just Measure?

It's worth pausing for a moment to think about exactly what we've measured in this experiment. Was it the effect of race on hiring? Or the difference in the experience of the average White job applicant from the average Black job applicant?

Well... no. What we have measured in this experiment is **just** the effect of having a Black-sounding name (as opposed to a White-sounding name) on your resume on the likelihood of getting a followup call from someone hiring in Boston or Chicago given identical resumes. In that sense, what we've measured is a small *piece* of the difference in the experience of Black and White Americans when seeking employment. As anyone looking for a job knows, getting a call-back is obviously a crucial step in getting a job, so this difference—even if it's just one part of the overall difference—is remarkable.

In [33]:
results

{'ex2_pvalue_female': 0.38,
 'ex2_pvalue_computerskills': 0.03,
 'ex2_pvalue_yearsexp': 0.85,
 'ex3_pvalue_education': 0.49,
 'ex4_validity': 'internal',
 'ex5_white_advantage_percent': 49.68,
 'ex5_white_advantage_percentage_points': 3.2,
 'ex5_pvalue': 4e-05,
 'ex6_black_pvalue': 4e-05,
 'ex8_black_nocollege': -4.05,
 'ex8_black_college': -2.81,
 'ex8_college_heterogeneity': 'less discrimination',
 'ex9_gender_and_discrimination': 'greater discrimination for women',
 'ex10_experiment_v_us': 'greater'}

In [34]:
assert set(results.keys()) == {
    "ex2_pvalue_computerskills",
    "ex2_pvalue_female",
    "ex2_pvalue_yearsexp",
    "ex3_pvalue_education",
    "ex4_validity",
    "ex5_pvalue",
    "ex5_white_advantage_percent",
    "ex5_white_advantage_percentage_points",
    "ex6_black_pvalue",
    "ex8_black_college",
    "ex8_black_nocollege",
    "ex8_college_heterogeneity",
    "ex9_gender_and_discrimination",
    "ex10_experiment_v_us",
}